In [141]:
'''
Jiayou Qin 03/18/2024
SIFT based analyzer
'''

import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
 
# video_path = "../gpt/Street normal.mp4" #change this file name as needed
video_path = "rotationTest.mp4"
cap = cv.VideoCapture(video_path)


visual_intensity = 1
visualize_points = False
visualize_cursor = True

for i in range(50): #skip frames, must be larger than 1
    success, frame = cap.read()
# frame = cv.rotate(frame, cv.ROTATE_90_CLOCKWISE)
sift = cv.SIFT_create()
frame = cv.resize(frame,(800,600))
y,x = frame.shape[:2]
center = np.array([x/2,y/2], dtype=float)
cursor = center.copy()

last_frame = frame
kp, des = sift.detectAndCompute(frame,None)
cv.namedWindow('p1_p2',cv.WINDOW_NORMAL)
cv.namedWindow('trans_p2',cv.WINDOW_NORMAL)
cv.imshow('p1_p2', frame)
cv.waitKey(0) #press key to begin

rot_m = np.array((2,3),)

last_m = None
while True:
    for i in range(5):
        success, frame = cap.read()
    if not success:
        break
    angles = np.array([],dtype=float)
    delta = np.array([],dtype=float)
    p1s = np.array([],dtype=float)
    p2s = np.array([],dtype=float)
    frame = cv.resize(frame,(800,600))
#     frame = cv.rotate(frame, cv.ROTATE_90_CLOCKWISE)
    frame_trans_p2 = frame.copy()
    out_frame = frame.copy()
    if not success:
        break
        
    kp_,des_ = kp,des
    kp, des = sift.detectAndCompute(frame,None)
    bf = cv.BFMatcher()
    matches = bf.knnMatch(des,des_,k=2)
    # Apply ratio test
    good = []
    for m,n in matches:
        if m.distance < 0.75*n.distance:
            p1 = np.array(kp[m.queryIdx].pt, dtype=float)
            p2 = np.array(kp_[m.trainIdx].pt, dtype=float)
            p1s = np.insert(p1s, 0, p1) #delta vector, indicating pixel change
            p2s = np.insert(p2s, 0, p2)
            
    delta = p1s - p2s
    p1s = p1s.reshape(-1,2)
    p2s = p2s.reshape(-1,2)

    
    
    trans_m,_ = cv.estimateAffine2D(p1s,p2s)
    # apply affine transformation to points in the last frame to estimate camera motions, result is stable
    p1s_trans = cv.transform( np.array([p1s]),trans_m)[0] 
    #pure motion movement and noise
    delta = p2s - p1s_trans  
    #use mean value of magnitude to filter out large noise
    magnitudes = np.sqrt((delta ** 2).sum(-1))[..., np.newaxis] 
    mean = np.mean(magnitudes)
    mask = (magnitudes > mean*5)[:,0]
    
    p1s = np.delete(p1s,mask,0)
    p2s = np.delete(p2s,mask,0)
    p1s_trans = np.delete(p1s_trans,mask,0)

    for i in range(len(p1s)):
        p1, p1_trans,p2 = p1s[i],p1s_trans[i],p2s[i]
        cv.circle(frame_trans_p2, p2.astype(int),2, (0,255,255) )
        cv.arrowedLine(frame_trans_p2,p1_trans.astype(int), p2.astype(int), (0,0,255),1)
        cv.arrowedLine(frame, p1.astype(int),p2.astype(int), (0,0,255),1)

    
    
    trans = cv.warpAffine(frame, trans_m, frame.shape[:2][::-1])
    cv.imshow('trans_p2', frame_trans_p2)
    cv.imshow('p1_p2', frame)
#     cv.imshow('vector field', out_frame)1
    k = cv.waitKey(5)
    if k ==27:
        break
    
    last_frame = frame
    
#     break;
cap.release()
cv.destroyAllWindows()


In [123]:
p1s.shape

p.shape

(967, 2)

In [121]:
p1s.shape

(989, 2)